In [1]:
import numpy as np
import pandas as pd
import string
import email
import os ##use this to traverse folders
from email.parser import Parser

#extract labels from trec06p/full/ file lists
with open('trec06p/full/index', 'r') as ifile:
    raw_labels = ifile.readlines()

#using file list to give labels and e-mail numbers

categories = [i.split( )[0] for i in raw_labels]
email_nums = [i.split(".")[-1][:-1] for i in raw_labels]
labels = pd.Series(categories, index=email_nums)


In [13]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [22]:
def info_of_email_2():
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    
    parser = Parser()
    info = []
    for nums in range(len(categories)):
        filepath = 'trec06p%s' % email_nums[nums]
        
        with open(filepath, 'r',) as ifile:
            email = parser.parse(ifile)
            info.append([email.get("From"), email.get('To'), email.get('cc'), email.get('bcc'), 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), " ".join([i for i in walkMsg(email) if i != None]),categories[nums]])
    return info

In [23]:
df = pd.DataFrame(info_of_email_2(),
                  columns=['From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [24]:
df.head()

,From,To,cc,bcc,Content_type,Subjects,Content,Classification
0,jhpb@sarto.budd-lake.nj.us (Joseph H. Buehler),None,None,None,text,new Catholic mailing list now up and running,The mailing list I queried about a few weeks a...,ham
1,"""Stella Lowry"" <rookcuduq@yahoo.com>","""Brian"" <bernice@groucho.cs.psu.edu>",None,None,multipart/alternative,re[12]:,\n ...,spam
2,"""Walter"" <trwmpca@downtowncumberland.com>",<arline@groucho.cs.psu.edu>,None,None,text/plain,Take a moment to explore this.,Academic Qualifications available from prestig...,spam
3,Scott Schwartz <schwartz@groucho.cs.psu.edu>,9fans <plan9-fans@cs.psu.edu>,None,None,None,Greetings,Greetings all. This is to verify your subscri...,ham
4,"""Mr Jailyn Koepke"" <kiflsbizc@attheworld.com>",melvin@groucho.cs.psu.edu,"jenna@groucho.cs.psu.edu, keri@groucho.cs.psu....",None,multipart/alternative,LOANS @ 3.17% (27 term),try chauncey may conferred the luscious not co...,spam


In [25]:
##encode
def robust_decode(bs):
    '''Takes a byte string as param and convert it into a unicode one.
First tries UTF8, and fallback to Latin1 if it fails'''
    cr = None
    try:
        cr = bs.decode('utf8')
    except UnicodeDecodeError:
        cr = bs.decode('ISO-8859-1')
    return cr
df['Content'] =[robust_decode(i) for i in df['Content']]
##remove newlines break/return
df['Content']=[i.replace('\n','').replace('\r','').replace('\t','') for i in df['Content']]

In [ ]:
##remove html tag
#import lxml.html

#df[['html' in i for i in df['Content_type']]]['Content'] = [lxml.html.fromstring(k).text_content() for k in df[['html' in i for i in df['Content_type']]]['Content']]

In [40]:
df.loc[0,"Content"]

u'The mailing list I queried about a few weeks ago is now up andrunning.  I have also set up an archive server; see below.  Thefollowing is the official welcome to the list message at the moment.Joe Buehler--------------------This mailing list is for people who desire serious, orthodoxdiscussion of the Roman Catholic religion.  I assume that it willcater mainly to Catholics, but everyone else is welcome, provided theyoperate within the below guidelines.  My own interests have adoctrinal bent, but I\'m certainly not going to limit this list to justthat sort of discussion.Having participated in USENET religion groups for about 5 years now,one of my primary observations about Catholics on the net is that theydo not know their religion very well.  My hope is that this list mighthelp remedy this problem to some extent.  I would like to make this anet resource available for Catholics who want to know more about theirreligion.As far as moderation policy goes: The Catholic Church is not ademoc

In [36]:
df.head()

,From,To,cc,bcc,Content_type,Subjects,Content,Classification
0,jhpb@sarto.budd-lake.nj.us (Joseph H. Buehler),None,None,None,text,new Catholic mailing list now up and running,The mailing list I queried about a few weeks a...,ham
1,"""Stella Lowry"" <rookcuduq@yahoo.com>","""Brian"" <bernice@groucho.cs.psu.edu>",None,None,multipart/alternative,re[12]:,...,spam
2,"""Walter"" <trwmpca@downtowncumberland.com>",<arline@groucho.cs.psu.edu>,None,None,text/plain,Take a moment to explore this.,Academic Qualifications available from prestig...,spam
3,Scott Schwartz <schwartz@groucho.cs.psu.edu>,9fans <plan9-fans@cs.psu.edu>,None,None,None,Greetings,Greetings all. This is to verify your subscri...,ham
4,"""Mr Jailyn Koepke"" <kiflsbizc@attheworld.com>",melvin@groucho.cs.psu.edu,"jenna@groucho.cs.psu.edu, keri@groucho.cs.psu....",None,multipart/alternative,LOANS @ 3.17% (27 term),try chauncey may conferred the luscious not co...,spam
